In [1]:
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Data-Importer") \
    .enableHiveSupport() \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# create database time_series
spark.sql("DROP TABLE IF EXISTS time_series;")
spark.sql("CREATE TABLE IF NOT EXISTS time_series\
(id bigint,time TIMESTAMP,lon float,lat float)\
PARTITIONED BY(area int)\
ROW FORMAT DELIMITED\
;")

# create database job
spark.sql("DROP TABLE IF EXISTS job;")
spark.sql("CREATE TABLE IF NOT EXISTS job\
(id bigint,start_time TIMESTAMP,end_time TIMESTAMP);")

# 创建输出表
CREATE TABLE output(id SERIAL PRIMARY KEY,job_id int4);


24/02/06 12:33:18 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


DataFrame[]

In [3]:
import csv
import random
import datetime
import random
import uuid
import time

In [4]:


num_points=100

def generate_id():
    start_num = [110,111,112,113,114,115,116,117,118,119,120]
    random_pn = random.randrange(0, len(start_num))
    random_num = str(random.randint(0, 99999999))
    id = "{}{:0<8}".format(start_num[random_pn], random_num)
    return id


def generate_utc():
    current_time = datetime.datetime.now()
    # utc = current_time.strftime("%Y-%m-%d %H:%M:%S")
    return int(round(current_time.timestamp()))

    
def generate_coordinates():
    longitude=round(random.uniform(130, 140), 7)
    latitude = round(random.uniform(45, 50), 7)
    return longitude,latitude


def get_area():
    return random.randint(0, 10)

for _ in range(num_points):
    id=generate_id()
    utc=generate_utc()
    lon,lat=generate_coordinates()
    area = get_area()
    sql = f'''INSERT INTO time_series  \
    PARTITION (area={area})\
    VALUES ({id},cast({utc} as timestamp), {lon}, {lat});\n'''\
    .format(id=id).format(utc=utc).format(lon = lon).format(lat = lat).format(area = area)
    spark.sql(sql)
print(f"Insert {num_points} rows data")


Insert 100 rows data


In [13]:
# 插入 job 表数据
def generate_utc_start():
    date_string = "2022-01-01 10:00:00"
    timestamp = time.strptime(date_string, "%Y-%m-%d %H:%M:%S")
    return int(time.mktime(timestamp))


def generate_utc_end():
    date_string = "2025-01-01 10:00:00"
    timestamp = time.strptime(date_string, "%Y-%m-%d %H:%M:%S")
    return int(time.mktime(timestamp))

utc_st = generate_utc_start()
utc_end = generate_utc_end()
sql = f"INSERT INTO job (id, start_time, end_time) VALUES (1, cast({utc_st} as timestamp), cast({utc_end} as timestamp));"
print(sql)
spark.sql(sql)


INSERT INTO job (id, start_time, end_time) VALUES (1, cast(1641031200 as timestamp), cast(1735725600 as timestamp));


DataFrame[]

In [5]:
# 查看数据量
result = spark.sql("SELECT COUNT(*) as cnt FROM time_series")
result.show()

+---+
|cnt|
+---+
|100|
+---+



In [6]:
# 查看一条数据
result = spark.sql("SELECT * FROM time_series LIMIT 1")
result.show()

+-----------+-------------------+---------+---------+----+
|         id|               time|      lon|      lat|area|
+-----------+-------------------+---------+---------+----+
|11842012002|2024-02-06 12:33:44|135.57184|45.837658|   2|
+-----------+-------------------+---------+---------+----+



In [7]:
# 查看分区数目
result = spark.sql("SHOW PARTITIONS time_series;")
result.show()

+---------+
|partition|
+---------+
|   area=0|
|   area=1|
|  area=10|
|   area=2|
|   area=3|
|   area=4|
|   area=5|
|   area=6|
|   area=7|
|   area=8|
|   area=9|
+---------+



In [9]:
result = spark.sql("SELECT version();")
result.show()

+--------------------+
|           version()|
+--------------------+
|3.3.3 8c2b3319c67...|
+--------------------+

